# Common Uses and Use Modes
This notebook demonstrates *proposed* VR usage styles


LB: store gene symbol/id
LB: variation type, other data "just for convenience"


In [37]:
from ga4gh.core import ga4gh_identify
from ga4gh.vrs import models
from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy
from ga4gh.vrs.extras.translator import Translator

seqrepo_rest_service_url = "http://localhost:5000/seqrepo"

data_proxy = SeqRepoRESTDataProxy(base_url=seqrepo_rest_service_url)
tlr = Translator(data_proxy=data_proxy)

# To generate unique identifiers

In [38]:
a = tlr.from_hgvs("NC_000013.11:g.32936732G>A")

In [39]:
str(a._id)

'ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-'

# As a storage or representation mechanism

In [40]:
import json
import redis
db = redis.Redis()

In [41]:
a = tlr.from_hgvs("NC_000013.11:g.32936732G>A")
a_id = str(a._id)
a_id

'ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-'

In [42]:
a_data = a.as_dict()
a_data

{'_id': 'ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-',
 'location': {'interval': {'end': 32936732,
   'start': 32936731,
   'type': 'SimpleInterval'},
  'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'type': 'SequenceLocation'},
 'state': {'sequence': 'A', 'type': 'SequenceState'},
 'type': 'Allele'}

In [43]:
# serialize object as json string, save to redis database
db.set(a_id, json.dumps(a_data))

True

In [44]:
# read json string from db, parse to dict
a2_data = json.loads(db.get(a_id))
a2_data

{'_id': 'ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-',
 'location': {'interval': {'end': 32936732,
   'start': 32936731,
   'type': 'SimpleInterval'},
  'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'type': 'SequenceLocation'},
 'state': {'sequence': 'A', 'type': 'SequenceState'},
 'type': 'Allele'}

In [45]:
a2 = models.Allele(**a2_data)

In [46]:
a == a2

True

In [57]:
tlr.to_hgvs(a2)

['CM000675.2:g.32936732C>A',
 '2oDBty0yKV9wHo7gg+Bt+fPgi5o:g.32936732C>A',
 'ff4c22faaa03aefada9b5e795267120becc0e592b87e92d3:g.32936732C>A',
 'SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT:g.32936732C>A',
 'NC_000013.11:g.32936732C>A',
 'GS__0wi-qoDrvram155UmcSC-zA5ZK4fpLT:g.32936732C>A',
 'a5437debe2ef9c9ef8f3ea2874ae1d82:g.32936732C>A',
 'da80c1b72d32295f701e8ee083e06df9f3e08b9a:g.32936732C>A']

## Associated with Annotations

## Embedded in other objects

In [47]:
patient_report = {
    "demo": {
        "name": "Reece Hart",
    },
    "observed-variation": [
        a.as_dict()
    ],
    "clinical-findings": [
        {
            "vr_id": a._id._value,
            "name": "bad_humor",
            "severity": "he's got it bad",
            "prognosis": "will never be funny"
        }
    ],
    "display-names": {
        a._id._value: {
            "hgvs": [
                "NC_000013.11:g.32936732G>A"
            ],
            "dbsnp": [
                "rs9999999999"
            ]
        }
    }
}

In [48]:
print(json.dumps(patient_report, indent=2))

{
  "demo": {
    "name": "Reece Hart"
  },
  "observed-variation": [
    {
      "_id": "ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-",
      "location": {
        "interval": {
          "end": 32936732,
          "start": 32936731,
          "type": "SimpleInterval"
        },
        "sequence_id": "ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT",
        "type": "SequenceLocation"
      },
      "state": {
        "sequence": "A",
        "type": "SequenceState"
      },
      "type": "Allele"
    }
  ],
  "clinical-findings": [
    {
      "vr_id": "ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-",
      "name": "bad_humor",
      "severity": "he's got it bad",
      "prognosis": "will never be funny"
    }
  ],
  "display-names": {
    "ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-": {
      "hgvs": [
        "NC_000013.11:g.32936732G>A"
      ],
      "dbsnp": [
        "rs9999999999"
      ]
    }
  }
}
